In [ ]:
import findspark
findspark.init('/spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping


spark = SparkSession.builder.appName("movielens").getOrCreate()

# Consultas sobre Movielens

## Schema de Movielens

![Schema](movielens_schema.png)

## ¿Cuántas películas calificó cada usuario?

Proceso:

1. Cargar la tabla ratings
1. Cargar la tabla movies
1. Unir ambas tablas
1. Agrupar por usuario
1. Contar las calificaciones


### Cargar las tablas

In [ ]:
# Tabla Ratings
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("./ml-latest-small/ratings.csv")\
    .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')

# Tabla Movies
movies_schema  = StructType(fields=[
    StructField("movieId",IntegerType(),True), 
    StructField("title",StringType(),True),
    StructField("genres",StringType(),True)
])

moviesDf = spark.read\
    .option("header", True)\
    .schema(movies_schema)\
    .csv("./ml-latest-small/movies.csv")

moviesDf = moviesDf.withColumn("genresSplit", split(moviesDf["genres"],"\|"))\
                        .drop('genres').withColumnRenamed("genresSplit","genres")\
                            .withColumn(\
                                "year",\
                                regexp_extract(\
                                           moviesDf["title"],\
                                           "^.+\(([0-9]+)\)$",\
                                           1)\
                                .cast(IntegerType()))\
                            .withColumn(\
                            "title_temp",\
                            regexp_extract(\
                                           moviesDf["title"],\
                                           "^(.+?) \([0-9]+\)$",\
                                           1))\
                            .drop('title')\
                        .withColumnRenamed("title_temp","title")

### Unir Ambas tablas

In [ ]:
movie_ratingsDF = ratingsDf.join(moviesDf,on="movieId",how="inner")
movie_ratingsDF.show()

### Agrupar por usuario

In [ ]:
from pyspark.sql.functions import grouping

movie_ratingsDF_grouped_by_userid = movie_ratingsDF.groupBy("userId")

### Contar las calificaciones

In [ ]:
movie_ratingsDF_grouped_by_userid.agg({"*": "count"}).collect()

In [ ]:
# Alternativa: metodo count
movie_ratingsDF_grouped_by_userid.count().show(10)

Existen muchas funciones que se pueden utilizar en agregaciones, la lista está en [Grouping](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/grouping.html)

## Filtrado

Podemos filtrar por cualquier columna.

Filtrado por usuario:

In [ ]:
movie_ratingsDF.filter(movie_ratingsDF.userId == 148).show()

In [ ]:
movie_ratingsDF.filter("userId = 148").show()

Mostrar solo ciertas columnas:

In [ ]:
movie_ratingsDF\
    .filter(movie_ratingsDF.userId == 148)\
    .select("userId","title")\
    .show()

Otras formas de agregar datos:

Calcular el promedio de datos

In [ ]:
movie_ratingsDF\
    .filter(movie_ratingsDF.userId == 148)\
    .select(avg("rating"))\
    .show()


Cual es el máximo rating que da el usuario 148?

In [ ]:
movie_ratingsDF\
    .filter(movie_ratingsDF.userId == 148)\
    .select(max("rating"))\
    .show()

En cuales peliculas?

In [ ]:
# Encadenar Filtros
movie_ratingsDF\
    .filter(movie_ratingsDF.userId == 148)\
    .filter(movie_ratingsDF.rating == 5.0)\
    .select(movie_ratingsDF.title)\
    .show()

In [ ]:
#Sintaxis SQL

movie_ratingsDF\
    .filter("userId = 148 AND rating = 5.0")\
    .select(movie_ratingsDF.title)\
    .show()

### Pivot

Algo muy poderoso son las *pivot tables* que permiten analizar datos facilmente.

In [ ]:
movie_ratingsDF.filter("movieId = 1197 AND date <= '19970930'").groupBy("userId").pivot("date").agg(avg("rating")).show()